In [28]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import MultilayerPerceptronClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml import Pipeline

In [29]:
spark = SparkSession.builder.appName("MLP with MLlib in PySpark").getOrCreate()

In [30]:
data = spark.read.format("csv").option("header", "true").load("./ml_model/fraudulent_transactions_anonymous_ia.csv")

In [31]:
# Convert target column to numeric using StringIndexer
label_indexer = StringIndexer(inputCol="is_fraud", outputCol="label")

# Convert string columns to numeric using StringIndexer
indexers = [
    StringIndexer(inputCol=col, outputCol=col+"_index", handleInvalid="keep")
    for col in ['amount', 'customer_id_avrge_amount_1day', 'customer_id_avrge_amount_1week',
                'customer_id_avrge_amount_1month', 'customer_id_avrge_amount_3month',
                'customer_id_count_1day', 'customer_id_count_1week', 'customer_id_count_1month',
                'customer_id_count_3month', 'account_id_avrge_amount_1day', 'account_id_avrge_amount_1week',
                'account_id_avrge_amount_1month', 'account_id_avrge_amount_3month',
                'account_id_count_1day', 'account_id_count_1week', 'account_id_count_1month',
                'account_id_count_3month', 'transaction_in_weekend', 'transaction_at_night']
]

# Create a VectorAssembler to assemble the feature columns
feature_columns = [col+"_index" for col in ['amount', 'customer_id_avrge_amount_1day', 'customer_id_avrge_amount_1week',
                                            'customer_id_avrge_amount_1month', 'customer_id_avrge_amount_3month',
                                            'customer_id_count_1day', 'customer_id_count_1week',
                                            'customer_id_count_1month', 'customer_id_count_3month',
                                            'account_id_avrge_amount_1day', 'account_id_avrge_amount_1week',
                                            'account_id_avrge_amount_1month', 'account_id_avrge_amount_3month',
                                            'account_id_count_1day', 'account_id_count_1week', 'account_id_count_1month',
                                            'account_id_count_3month', 'transaction_in_weekend', 'transaction_at_night']
]
vectorAssembler = VectorAssembler(
    inputCols=feature_columns,
    outputCol="features"
)
# Apply the VectorAssembler to the data
#data = vectorAssembler.transform(data)

In [32]:
# Create a Multilayer Perceptron Classifier
layers = [len(feature_columns), 10, 5, 2]  # Adjust the number of neurons in each layer as needed
mlp = MultilayerPerceptronClassifier(
    labelCol="label",
    featuresCol="features",
    layers=layers,
    seed=42
)

In [34]:
# Create a pipeline to combine the stages
pipeline = Pipeline(stages=[label_indexer] + indexers + [vectorAssembler, mlp])

# Split the data into training and testing sets
train_data, test_data = data.randomSplit([0.8, 0.2], seed=42)

# Fit the pipeline on the training data
model = pipeline.fit(train_data)

Py4JJavaError: An error occurred while calling o2707.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 5 in stage 293.0 failed 1 times, most recent failure: Lost task 5.0 in stage 293.0 (TID 686) (pyspark executor driver): java.lang.OutOfMemoryError: Java heap space

Driver stacktrace:
	at org.apache.spark.scheduler.DAGScheduler.failJobAndIndependentStages(DAGScheduler.scala:2672)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$abortStage$2(DAGScheduler.scala:2608)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$abortStage$2$adapted(DAGScheduler.scala:2607)
	at scala.collection.mutable.ResizableArray.foreach(ResizableArray.scala:62)
	at scala.collection.mutable.ResizableArray.foreach$(ResizableArray.scala:55)
	at scala.collection.mutable.ArrayBuffer.foreach(ArrayBuffer.scala:49)
	at org.apache.spark.scheduler.DAGScheduler.abortStage(DAGScheduler.scala:2607)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$handleTaskSetFailed$1(DAGScheduler.sca

In [ ]:
# Make predictions on the testing data
predictions = model.transform(test_data)

# Evaluate the model
evaluator = MulticlassClassificationEvaluator(
    labelCol="label",
    predictionCol="prediction",
    metricName="accuracy"
)

In [ ]:
accuracy = evaluator.evaluate(predictions)
print("Accuracy: {:.2f}%".format(accuracy * 100))